[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ahmedlone127/Multilingual_Sentiment/blob/master/Colab/multilingual_sentiment.ipynb)##1. Install Spark NLP 

In [ ]:
! wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

import sparknlp
spark = sparknlp.start(gpu = True)
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline
import pandas as pd
print("Spark NLP version", sparknlp.version())
print("Apache Spark version:", spark.version)

spark

--2021-11-26 13:07:28--  http://setup.johnsnowlabs.com/colab.sh
Resolving setup.johnsnowlabs.com (setup.johnsnowlabs.com)... 51.158.130.125
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://setup.johnsnowlabs.com/colab.sh [following]
--2021-11-26 13:07:28--  https://setup.johnsnowlabs.com/colab.sh
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh [following]
--2021-11-26 13:07:30--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:44

##2. Load Data

In [ ]:
import pandas as pd 
df = pd.read_csv("Sentiment_Data.csv")
df = spark.createDataFrame(df)
from sklearn.model_selection import train_test_split
train_df , test_df = df.randomSplit([0.8, 0.2])
train_df.show()

+--------+---------------------------------+
|       y|                             text|
+--------+---------------------------------+
|negative|                  一个 人 害怕 咩|
|negative|  一个 人 快活 两个 人 生活 三...|
|negative|                一个 人 觉得 难过|
|negative|             一个 人 过年 好 无聊|
|negative|                一个 人 难受 算了|
|negative|      一个 人好 无聊 陪 聊聊天 吧|
|negative|                 一个 低沉 机器人|
|negative|      一个 朋友 生气 说 改 怎么办|
|negative|                   一个 机器 骂人|
|negative|           一个 比 一个 浪费 土地|
|negative|                一个 比 一个 猥琐|
|negative|一个 特别 讨厌 人 他 每次 坏 好事|
|negative|   一个 疼 爱 对 又 这么 痴情 ...|
|negative|         一个 讨厌 人 他 老 找 事|
|negative|            一个月 七天 就够 心烦|
|negative|                 一些 感叹 卡西欧|
|negative|               一到 四大美女 麻烦|
|negative|                一哥 什么 不好 呢|
|negative|              一大早 上 我够 无聊|
|negative|                   一大早 忒 无聊|
+--------+---------------------------------+
only showing top 20 rows



#3. Train Model

In [ ]:
from pyspark.ml import Pipeline
from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *


document = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")
sent_embeddings = BertSentenceEmbeddings.pretrained("sent_bert_use_cmlm_multi_base_br", "xx") \
      .setInputCols("document") \
      .setOutputCol("sentence_embeddings")
classifierDl = ClassifierDLApproach()\
  .setInputCols(["sentence_embeddings"])\
  .setOutputCol("class")\
  .setLabelColumn("y")\
  .setMaxEpochs(125)\
  .setLr( )  
pipeline = Pipeline(
    stages = [
        document,
        sent_embeddings,
        classifierDl
    ])
mutli_lingual_model = pipeline.fit(train_df)


sent_bert_use_cmlm_multi_base_br download started this may take some time.
Approximate size to download 1.7 GB
[OK!]


## 4. Save the Model

In [ ]:
mutli_lingual_model.stages[-1].write().overwrite().save('/content/tmp_multi_classifierDL_model')


#5. Evaluate Trained Model on Test Data

In [ ]:
preds = mutli_lingual_model.transform(test_df)
preds.show()

+--------+---------------------------------+--------------------+--------------------+--------------------+
|       y|                             text|            document| sentence_embeddings|               class|
+--------+---------------------------------+--------------------+--------------------+--------------------+
|negative|  一个 人 玩 无聊 加 寂寞 所以...|[[document, 0, 21...|[[sentence_embedd...|[[category, 0, 21...|
|negative|    一个 机器人 说话 你好 无聊 哎|[[document, 0, 16...|[[sentence_embedd...|[[category, 0, 16...|
|negative|    一会 走神 一会 故障 怎么 服务|[[document, 0, 16...|[[sentence_embedd...|[[category, 0, 16...|
|negative|        一加 什么 时候 下 等于 三|[[document, 0, 14...|[[sentence_embedd...|[[category, 0, 14...|
|negative| 一向 都 很淡 定 可是 关键时刻...|[[document, 0, 27...|[[sentence_embedd...|[[category, 0, 27...|
|negative|  一天 不 贱人 犯贱 活不了 够 ...|[[document, 0, 20...|[[sentence_embedd...|[[category, 0, 20...|
|negative|      一时 说不出来 说 玩 什么 吧|[[document, 0, 15...|[[sentence_embedd...|[[category, 0, 15...|

In [ ]:
df = preds.select(['y','class.result']).toPandas()
df['result'] = df['result'].apply(lambda x : x[0])
df

,y,result
0,negative,negative
1,negative,negative
2,negative,negative
3,negative,positive
4,negative,positive
...,...,...
5904,positive,negative
5905,positive,positive
5906,positive,positive
5907,positive,positive


In [ ]:

# We are going to use sklearn to evalute the results on test dataset
from sklearn.metrics import classification_report
print(classification_report(df['result'], df['y']))

              precision    recall  f1-score   support

    negative       0.89      0.89      0.89      2434
    positive       0.92      0.92      0.92      3475

    accuracy                           0.91      5909
   macro avg       0.91      0.91      0.91      5909
weighted avg       0.91      0.91      0.91      5909

